In [1]:
%matplotlib inline

import os,sys
import numpy as np
import healpy as hp
import pandas as pd
import astropy.io.fits as fits
import matplotlib
from matplotlib import rc
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output
rc('text', usetex=True)
font = {'weight' : 'bold',
        'size'   : 22}
matplotlib.rc('font', **font)

sys.path.insert(1, '/home/luiz/IC/Codes/Noise_Debias/scripts')
import gmca4im_lib2 as g4i
import Extension4BINGO as cs

In [14]:
#######################################################
################   GENERAL INFORMATIONS   #############
#######################################################
wtransform       = "starlet"
maps_wout_mean   = True
apply_mask       = False
add_noise        = False
#######################################################
################   WAVELETS PARAMETERS   ##############
#######################################################
J     = 3  #number of scales
use_c = True  # if you will use wavelet scale in the analysis
##############
# If you to use wtransforms by S2Let code, please, fill in the variables below:
L        = None #If you write "None", it will use L=3*nside
J_min    = 1
B        = 10       
N        = 3  # Number of directions (This is for Directional only)
spin     = 0  # set to 0 for temperature. if non-zero, plotting routines must be changed! (This is for Directional only)
upsample = 0  # 1 means all scales at full resolution L # 0 means multiresolution wavelet transform (This is for Directional only)

# In the S2LET code, J scales is defined by code and not by J above.
#######################################################
#PARAMETROS ENTRADA 
######################################################################################################
########FastICA parametros de entrada da função
n = 3
whiten = True
fun = 'logcosh' 
max_iter = 100
tol = 0.0001
div          = 1 #  J+1  #J/div will should be even number
without_covx = True # if your mixmatrix estimated will use covariance matrix of the observer data with ponderation
######################################################################################################
#######################################################
################   DEBIAS PARAMETERS   ################
#######################################################
seed_used = 10
#######################################################
################   PATHS PARAMETERS   #################
#######################################################
#path outputs
pathout       = "/home/luiz/IC/Datas_Maps/Cls/Cls_starlet" #Put here your path to the output cls
cl_type_save  = "reconstruction" #You should choice between reconstruction or residuals cls values
#######################################################
################   NAME FILES PARAMETERS   ############
#######################################################
# Name of FITS files inside of the pathmaps
name_mask = "Mask_tot256.fits" #put this file in the same directory of the other maps

#Directory names
dir_observed  = "/home/luiz/IC/Datas_Maps/Cubos_Input_L10_L25_White_Noise"
dir_mask      = "/home/luiz/IC/Datas_Maps/Mask"
dir_prior     = "/home/luiz/IC/Datas_Maps/Cubos_Prior_WN" 
dir_noise     = "/home/luiz/IC/Datas_Maps/wn_masked"         #Put here directory name of the noise maps 
dir_pure      = "/home/luiz/IC/Datas_Maps/Cubos_21cm_Masked" #Put here directory name of the pure maps 
dir_projprior = "/home/luiz/IC/Datas_Maps/Cubos_Prior_WN"    #Put here directory name of the prior maps
dir_projnoise = "/home/luiz/IC/Datas_Maps/wn_masked"         #Put here directory name of the noise maps
dir_projpure  = "/home/luiz/IC/Datas_Maps/Cubos_21cm_Masked" #Put here directory name of the pure maps 

In [15]:
params_maps = pd.Series({"without_mean":maps_wout_mean, "apply_mask":apply_mask, "add_noise":add_noise, "cl_type_save":cl_type_save})
params_CS   = pd.Series({"n":n, "whiten":whiten, "fun":fun, "max_iter":max_iter, "tol":tol, "division":div,"without_covx":without_covx, "seed_used":seed_used})
params_WT   = pd.Series({"wtransform":wtransform, "J":J, "use_c":use_c, "L":L, "Jmin":J_min, "B": B, "N":N, "spin":spin, "upsample":upsample})
params_path = pd.Series({"pathout":pathout, "dir_observed":dir_observed, "dir_mask":dir_mask, "dir_noise":dir_noise, "dir_prior":dir_prior,"dir_pure":dir_pure, "name_mask":name_mask})

### Foregrounds + HI + Noise
$C_{\ell}^{\textrm{FG+HI+N}}$

In [16]:
names = np.asarray(os.listdir(params_path.dir_observed))
#nseed = np.asarray([iname.split("_")[-2] for iname in names])
nseed = cs.extracting_seed_from_filenames(vectornames=names)

timei = time.time()
params_maps["getdata"] = "observed"
subdirs = cs.checkdir(params_path.pathout, subdirs=["21cm","foregrounds","mixmatrix"])
for i,iname in enumerate(names):
    time0 = time.time()
    params_path["name_observed"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    X = cs.maps2ICAmaps(X, params_WT, params_CS)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)
    del X
    time0   = time.time()-time0
    print("Loop time {0}: {1:.2f} min\n".format(i,time0/60))
clear_output(wait=True)    
time0 = time.time()-timei
print("Completed in: {:.2f} min".format(time0/60))

Completed in: 154.99 min


### Noise
$C_{\ell}^{\textrm{N}}$

In [17]:
names = np.asarray(os.listdir(params_path.dir_noise))
#nseed = np.asarray([iname.split("_")[-1][:2] for iname in names])#selecting L-number from string name
nseed = cs.extracting_seed_from_filenames(vectornames=names)

timei   = time.time()
params_maps["getdata"] = "noise"
subdirs = ["noise"]
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))
for i,iname in enumerate(names):
    time0 = time.time()
    params_path["name_noise"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    
    time0 = time.time()-time0
    print("[L{0}] Loop time: {1:.2f} min".format(nseed[i],time0/60))
clear_output(wait=True)    
time0 = time.time()-timei
print("Completed in: {:.2f} min".format(time0/60))

Completed in: 0.86 min


### Prior
$C_{\ell}^{\textrm{prior}}$

In [18]:
names = np.asarray(os.listdir(params_path.dir_prior))
#nseed = np.asarray([iname.split("_")[-1].split(".")[0][:3] for iname in names])#selecting L-number from string name
nseed = cs.extracting_seed_from_filenames(vectornames=names)

timei   = time.time()
params_maps["getdata"] = "prior"
subdirs = ["prior"]
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))
for i,iname in enumerate(names):
    time0 = time.time()
    params_path["name_prior"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    
    time0 = time.time()-time0
    print("[L{0}] Loop time: {1:.2f} min".format(nseed[i],time0/60))
clear_output(wait=True)    
time0 = time.time()-timei
print("Completed in: {:.2f} min".format(time0/60))

Completed in: 0.78 min


### 21cm pure
$C_{\ell}^{\textrm{pure}}$

In [19]:
names = np.asarray(os.listdir(params_path.dir_pure))
#nseed = np.asarray([iname.split("_")[-1].split(".")[0][:3] for iname in names])#selecting L-number from string name
nseed = cs.extracting_seed_from_filenames(vectornames=names)

timei   = time.time()
params_maps["getdata"] = "pure"
subdirs = ["pure"]
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))
for i,iname in enumerate(names):
    time0 = time.time()
    params_path["name_pure"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    
    time0 = time.time()-time0
    print("[L{0}] Loop time: {1:.2f} min".format(nseed[i],time0/60))
clear_output(wait=True)    
time0 = time.time()-timei
print("Completed in: {:.2f} min".format(time0/60))

Completed in: 0.78 min


### 21cm proj pure
$C_{\ell}^{\textrm{proj pure}}$

In [20]:
names = np.asarray(os.listdir(params_path.dir_pure))
#nseed = np.asarray([iname.split("_")[-1].split(".")[0][:3] for iname in names])#selecting L-number from string name
nseed = cs.extracting_seed_from_filenames(vectornames=names)

timei   = time.time()
params_maps["getdata"] = "pure"
subdirs = ["projpure"]
L0      = "L{}".format(params_CS.seed_used)
A       = cs.loadmixmatrix(params_path.pathout,"mixmatrix")
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))
for i,iname in enumerate(names):
    time0 = time.time()
    params_path["name_pure"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, A=A[L0], params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    
    time0 = time.time()-time0
    print("[L{0}] Loop time: {1:.2f} min".format(nseed[i],time0/60))
clear_output(wait=True)    
time0 = time.time()-timei
print("Completed in: {:.2f} min".format(time0/60))

Completed in: 0.86 min


### 21cm  proj noise
$C_{\ell}^{\textrm{proj noise}}$

In [21]:
names = np.asarray(os.listdir(params_path.dir_noise))
#nseed = np.asarray([iname.split("_")[-1][:2] for iname in names])#selecting L-number from string name
nseed = cs.extracting_seed_from_filenames(vectornames=names)
timei   = time.time()
params_maps["getdata"] = "noise"
subdirs = ["projnoise"]
L0      = "L{}".format(params_CS.seed_used)
A       = cs.loadmixmatrix(params_path.pathout,"mixmatrix")
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))
for i,iname in enumerate(names):
    time0 = time.time()
    params_path["name_noise"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, A=A[L0], params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    
    time0 = time.time()-time0
    print("[L{0}] Loop time: {1:.2f} min".format(nseed[i],time0/60))
clear_output(wait=True)    
time0 = time.time()-timei
print("Completed in: {:.2f} min".format(time0/60))

Completed in: 0.90 min


### 21cm  proj prior
$C_{\ell}^{\textrm{proj prior}}$

In [22]:
names   = np.asarray(os.listdir(params_path.dir_prior))
#nseed   = np.asarray([iname.split("_")[-1].split(".")[0][:3] for iname in names])#selecting L-number from string name
nseed = cs.extracting_seed_from_filenames(vectornames=names)

timei   = time.time()
params_maps["getdata"] = "prior"
subdirs = ["projprior"]
L0      = "L{}".format(params_CS.seed_used)
A       = cs.loadmixmatrix(params_path.pathout,"mixmatrix")
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))
for i,iname in enumerate(names):
    time0 = time.time()
    params_path["name_prior"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, A=A[L0], params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    
    time0 = time.time()-time0
    print("[L{0}] Loop time: {1:.2f} min".format(nseed[i],time0/60))
clear_output(wait=True)    
time0 = time.time()-timei
print("Completed in: {:.2f} min".format(time0/60))

Completed in: 0.89 min
